In [1]:
import pandas as pd
import re
import warnings
import json
from tqdm import tqdm
import numpy as np
import geopandas as gpd
import os
from glob import glob
from geopandas import GeoDataFrame
from shapely.geometry import Point

df_energy = pd.read_excel("./csv/신재생관련_농지전용필지(2013_2021)_221128_통합.xlsx"
df_energy['FULL_ADDR'] = df_energy['시도'] + " " +df_energy['시군구'] + " " + df_energy['농지소재지']

In [ ]:
# 파일 선택시 경로를 복사하는 방향으로 input()?
#file = './csv/신재생관련_농지전용필지(2013_2021)_221128_통합.xlsx'
#col_nm = "상세주소1"

# csv, excel -> 데이터 유형 필요, encoding -> utf8, cp949 유형 필요?, 해당 데이터는 header의 정보가 필요.. 이 부분도 구현?
#df = pd.read_excel(file, header=1,)
df_energy = df_energy.reset_index()
df_energy = df_energy.rename(columns = {'index':'고유번호'})

fix_sd_dict1 = {'서울 ': '서울특별시 ', '서울시': '서울특별시 ','부산 ': '부산광역시 ', '부산시':'부산광역시 ','대구 ': '대구광역시 ', '대구시':'대구광역시 ',
                '울산 ': '울산광역시 ', '울산시':'울산광역시 ','인천 ': '인천광역시 ','인천시':'인천광역시 ', '광주 ': '광주광역시 ',
                '대전 ': '대전광역시 ','대전시':'대전광역시 ', '세종 ': '세종특별자치시 ', '세종시':'세종특별자치시 ','경기 ': '경기도 ',
                '강원 ': '강원도 ', '충북': '충청북도 ', '충남': '충청남도 ',
                '전북': '전라북도 ', '전남': '전라남도 ', '경북 ': '경상북도 ',
                '경남': '경상남도 ', '제주 ': '제주특별자치도 ', '제주도': '제주특별자치도 '}


bjd_cd = pd.read_csv('./csv/법정동코드 전체자료.csv')


with open('bjd.json', 'r') as f:
    bjd = json.load(f)

with open('pnu.json', 'r') as f:
    pnu = json.load(f)

with open('road.json', 'r') as f:
    rtj = json.load(f)
    


In [ ]:
def postal_code(df, col_nm):
    '''
    우편번호 제거 및 좌측 공백 제거
    '''
    df[col_nm] = df[col_nm].str.lstrip('0123456789- ')
    return df

def remove_space(df, col_nm):
    '''
    이중 공백문자 제거
    '''
    df[col_nm] = df[col_nm].str.strip()
    df_pre = df[col_nm]
    while True:
        df[col_nm] = df[col_nm].apply(lambda x : x.replace('  ',' '))
        if not (df[col_nm] != df_pre).sum():
            break
        else:
            df_pre = df[col_nm]
    return df

def sd_fullname(df,col_nm):
    '''
    전역변수 fix_sd_dict1의 축약 시도명을 변경
    '''
    df[col_nm].fillna('', inplace=True)
    df[col_nm] = df[col_nm].replace(fix_sd_dict1,regex=True)
    # df[col_nm] = df[col_nm].replace(fix_sd_dict2,regex=True)
    return df

def replace_exception_char(df, col_nm):
    '''
    특수문자 및 예외문구들에 대한 전처리
    '''
    def repl(match):
        return match.group(1)
    df[col_nm] = df[col_nm].str.replace('null','') # null 문자 제거
    # df[col_nm] = df[col_nm].apply(lambda x: re.sub('외[\d|\s]+필지',' ', x))
    # df[col_nm] = df[col_nm].str.replace('산 ','산') # null 문자 제거 一-龥
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'[一-龥]+', '', x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'번지\s*(\d+)호', repl, x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'(\d+-\d+)호', repl, x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'[a-zA-Z]+동', '', x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'외[\d|\s]+[필지|건|개소]+', ' ', x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'\b주(\d+|,)+[동호]?\b', ' ', x))
    # df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'외[\d|\s]+필', ' ', x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'외[\d|\s]*\(', ',', x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'[\d|\s]*필지', ',', x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'\b(\d+\s*){1}(,+\s*\d+)+동\b', ',', x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'\b\d+동|번지', ',', x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'\b\d+평방메타\b', ',', x))
    # df['check'] = df[col_nm].apply(lambda x: True if len(re.sub(r'\b(\d+\s*){1}(,+\s*\d+)+동\b', x)) else False)
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'[.(&]',',', x))
    df[col_nm] = df[col_nm].apply(lambda x: re.sub(r'[^\w\s,-]', ' ', x))
    df[col_nm] = df[col_nm].apply(lambda x: double_comma(x))
    df = remove_space(df, col_nm)
    return df

def mk_sgg_space(df, col_nm, slist):
    '''
    법정동 list에 존재하는 시도, 시군, 구, 읍면, 동리가 각 지역명에 공백을 추가하는 기능
    ex) 충청남도천안시동남구 -> 충청남도 천안시 동남구
    '''

    # 리까지 포함하게되면 시간 소요가 3분정도 늘어나서 제외해놓았음.
    for s1 in tqdm(slist):
        s1_f = s1[-1]
        if len(s1) == 0 or s1_f == "리":
            continue
        '''
        인천광역시 강화군 교동면 교동동로 503-1 5-1호 / 안동시와 같은 경우도 교동 면, 교동 동로, 안동 시 같이 되어 아래와 같은 if문, pattern을 추가해줌 
        충청남도 천안시 동남구동면 화덕1길 201-13 화덕리304 등 
        처리안되는 경우들
        '''
        if s1_f == '군' or s1_f == '시' or s1_f == '도':
            df[col_nm] = df[col_nm].apply(lambda x: x.replace(s1, s1+" "))
            continue
        pattern = s1+"\d*\w{0,1}[시|읍|면|길|로|리|가|번|동|군|구]"
        regex = re.compile(pattern)
        df[col_nm] = df[col_nm].apply(lambda x: x if len(regex.findall(x)) else x.replace(s1, s1+" "))
    return df

def sep_bjd_address(df, col_nm):
    '''
    법정동코드 전체자료.csv의 형태가 각 지역명별로 나뉘지 않은 상태(ex, 서울특별시 금천구 가산동)이기 때문에 나누어주는 기능
    향후 발전방안) json파일로 변환 후 불러오는 방향으로 하여 해당 기능 삭제
    '''
    bjd_regex = r'(?P<SD_NM>^\b[가-힣]+[시도]\s*\b)(?P<SG_NM>\b[가-힣]+[시군]\s*\b)?(?P<GU_NM>\b[가-힣]+[구]\s*\b)?(?P<EM_NM>\b\w+[읍면]\s*\b)?(?P<DLG_NM>\b\w+[동리가]\b)'
    jb = df[col_nm].str.extract(bjd_regex)
    jb.fillna('', inplace=True)
    for c in jb.columns:
        jb[c] = jb[c].str.strip()
    return jb

def sep_name(df, bjd, col_nm):
    # 지명+" "(공백)
    '''
    법정동명을 나누는 기능 -> 하나의 set에 각 지역명 모두 저장 -> 지역명+공백 기능 수행 -> 이중 공백 제거
    '''
    
    bjd = bjd[bjd.폐지여부 == "존재"]
    bjd = sep_bjd_address(bjd, "법정동명")
    local_name_set = set()
    for name in bjd.columns:
        local_name_set.update(list(bjd[name].unique()))
    local_name_set.remove('')
    df = mk_sgg_space(df, col_nm, local_name_set)
    df = remove_space(df, col_nm)
    return df

def sep_address(df, col_nm,):
    '''
    도로명주소와 지번주소의 정규표현식을 각 정의하고, 이를 통해 상세주소1에서 매칭되는 모든 주소를 추출
    '''
    global bjd
    road_regex = r'(?P<road_sd>\b[가-힣]+(특별시|자치시|광역시|도)\s+\b)?(?P<road_sg>\b[가-힣]+[시군]\s+\b)?(?P<road_g>\b[가-힣]+[구]\s+\b)?(?P<road_em>\b\w+[읍면]\s+\b)?(?P<road_rg>\b\w+[로길]\s*)(?P<road_jb>([,|\s]*\d+-?\d*[^a-zA-Z가-힣]*)+)'
    jb_regex = r'(?P<sd>\b[가-힣]+(특별시|자치시|광역시|도)\s+\b)?(?P<sg>\b[가-힣]+[시군]\s+\b)?(?P<g>\b[가-힣]+[구]\s+\b)?(?P<em>\b\w+[읍면]\s+\b)?(?P<dlg>\b\w+(동\s|리|가\s)\s*)(?P<jb>([산|,|\s]*\d+-?\d*[^a-zA-Z가-힣]*)+)'
    extract = df[col_nm].str.extractall(road_regex+'|'+jb_regex)
    extract.fillna('', inplace=True)
    extract = extract[['road_sd', 'road_sg', 'road_g', 'road_em', 'road_rg', 'road_jb', 'sd', 'sg', 'g', 'em', 'dlg', 'jb']]
    for i in extract.columns:
        extract[i] = extract[i].str.strip()
    
    '''
    간단한 주소보완기능 -> 지번주소에서 해당 동(동리가)의 bjd length가 1일때 주소 보완
    '''
    # with open('./bjd.json', 'r') as f:
    #     bjd = json.load(f)

    n_list = []
    for x in tqdm(extract.iterrows()):
        x0, x1 = x[0], x[1]
        jb_sd, jb_sg, jb_g, jb_em, jb_dlg = x1.sd, x1.sg, x1.g, x1.em, x1.dlg
        if jb_dlg and bjd.get(jb_dlg):
            if len(bjd[jb_dlg]['nm']) == 1:
                jb_sd, jb_sg, jb_g, jb_em = bjd[jb_dlg]['nm'][0]['sd'], bjd[jb_dlg]['nm'][0]['sg'], bjd[jb_dlg]['nm'][0]['g'], bjd[jb_dlg]['nm'][0]['em']
        n_list.append([jb_sd, jb_sg, jb_g, jb_em])

    extract[['sd', 'sg', 'g', 'em']] = n_list

    # 고유번호 추가(MULTIINDEX)
    # extract.reset_index(names=['Main_id', 'Sub_id'], inplace=True)
    extract['id'] = extract.index
    return extract

def address_supplement(df,):
    '''
    주소보완기능
    전북 고창군 상하면 장호로 227 용정리 2148-1 와 같은 주소는 '전라북도 고창군 상하면 장호로 227'과 '전라북도 고창군 상하면 용정리 2148-1'과 같이 만들어주는 기능임.
    위와 같은 경우가 있을 때, 시도명, 시군명, 구명, 읍면명을 두 주소(전북 고창군 상하면 장호로 227 / 용정리 2148-1)에서 수집하여 
    다른 주소가 하나라도 있을 경우에는 패스(서울특별시 강남구 영동대로 210 / 전북 장수군 장수읍 수분리 900-3)하고 다른 주소가 없을 경우 보완해주는 방식
    '''
    road_col = ['road_sd', 'road_sg', 'road_g', 'road_em'] # 도로명주소 columns
    jb_col = ['sd', 'sg', 'g', 'em'] # 지번주소 columns
    improve_dict = dict()
    for i in tqdm(df['id'].apply(lambda x: x[0]).unique()):
        ndict = {'sd':set(), 'sg':set(), 'g':set(), 'em':set()}
        if len(df.loc[i]) == 1:
            continue
        for j in df.loc[i].iterrows():
            for k in zip(road_col, jb_col):
                if j[1][k[0]]:
                    ndict[k[1]].add(j[1][k[0]])
                if j[1][k[1]]:
                    ndict[k[1]].add(j[1][k[1]])
        if not (len(ndict['sd']) > 1 or len(ndict['sg']) > 1 or len(ndict['g']) > 1 or len(ndict['em']) > 1):
            improve_dict[i] = [x.pop() if x else '' for x in [ndict['sd'], ndict['sg'], ndict['g'], ndict['em']]]
    for i in tqdm(improve_dict):
        df.loc[i, road_col+jb_col] = improve_dict[i]+improve_dict[i]
    return df

def double_comma(str):
    '''
    예외처리 및 지번 분할단계에서 문자 및 공백을 ,로 치환했기 때문에 ,,인 경우를 ,로 치환하는 기능
    '''
    if str == str.replace(",,", ","):
        return str
    return double_comma(str.replace(",,", ","))

def sep_jb(jb):
    '''
    ['324-4, 359-62,359-69, 359-84,359-87, 359-95'] -> ['324-4', '359-62', '359-69', '359-84', '359-87', '359-95']
    '''
    jb = jb.replace("산 ", "산")    # 경상북도 영양군 입암면 삼산리 산 406-4 -> ['산', '406-4']
    jb = jb.replace(" ", ",")
    jb = double_comma(jb)
    jb = jb.split(",")
    return jb

def rm_space_str(text):
    '''
    양측 공백 제거 및 이중 공백 단일 공백으로 치환
    '''
    text = text.strip()
    if text == text.replace("  ", " "):
        return text
    return rm_space_str(text.replace("  ", " "))

def address_pnu(locallist:list,):
    global bjd, pnu
    bjd_cd=''
    dlg = locallist[4]
    match_col_nm = ['sd', 'sg', 'g', 'em', 'dlg']
    ms_add = locallist[5] + '-' + locallist[6]
    main, sub = locallist[5].rjust(4, '0'), locallist[6].rjust(4, '0')
    if bjd.get(dlg):
        if len(bjd[dlg]['nm']) == 1:
            bjd_cd = bjd[dlg]['nm'][0]['CD']
        else:
            match_list = []
            for i in range(len(bjd[dlg]['nm'])):
                for j, mcn in enumerate(match_col_nm):
                    if (bjd[dlg]['nm'][i][mcn] and locallist[j]) and (bjd[dlg]['nm'][i][mcn] != locallist[j]):
                        break
                    if j == len(match_col_nm)-1:
                        match_list.append(bjd[dlg]['nm'][i])
            if len(match_list) == 1:
                bjd_cd = match_list[0]['CD']
    if not bjd_cd or not pnu.get(bjd_cd):
        return None
    
    if pnu[bjd_cd].get(ms_add):
        san_cd = str(int(pnu[bjd_cd][ms_add])+1)
    else:
        san_cd = str(locallist[7]+1)
    pnu_cd = bjd_cd+san_cd+main+sub

    return pnu_cd

def jb_ms_sep(first, second, post_main):
    if first and second:
        post_main = first
        return first, second, post_main
    elif first and not second and post_main:
        first, second = post_main, first
        return first, second, post_main
    elif first and not second and not post_main:
        return first, '0', ''
    elif not first and second and post_main:
        return post_main, second, post_main
    elif not first and second and not post_main:
        return '', second, ''
    elif not first and not second:
        return '0', '0', ''

def road_to_jbaddress(locallist):
    global rtj
    
    roadnm = locallist[5]
    msjb = locallist[4]
    match_col_nm = ['sd', 'sg', 'g', 'em', 'roadjb']

    if rtj.get(roadnm):
        match_list = []

        for k, v in rtj[roadnm].items():
            for j, mcn in enumerate(match_col_nm):
                if j != 4 and (v[mcn] and locallist[j]) and (v[mcn] != locallist[j]):
                    break
                if j == 4 and not v[mcn].get(locallist[j]):
                    break
                if j == len(match_col_nm)-1:
                    match_list.append([k, v])

        if len(match_list) == 1:
            bjd_cd = match_list[0][0]
            vdict = match_list[0][1]
            main, sub = vdict['roadjb'][msjb][0][0].rjust(4, '0'), vdict['roadjb'][msjb][0][1].rjust(4, '0')
            pnucd = bjd_cd+str(int(vdict['산'])+1)+main+sub
            if vdict['roadjb'][msjb][0][1] != '0':
                fjb = vdict['roadjb'][msjb][0][0]+'-'+vdict['roadjb'][msjb][0][1]
            else:
                fjb = vdict['roadjb'][msjb][0][0]
            address = rm_space_str(' '.join([vdict['sd'], vdict['sg'], vdict['g'], vdict['em'], vdict['dlg'], fjb]))
            return address, pnucd
        else:
            return None, None

    else:
        return None, None

def make_multi_add(df,):
    '''
    각 주소에 지번이 여러개 존재하는 경우 주소 생성 기능
    ex) 충청남도 공주시 계룡면 화헌리 165-1 165-1, 165-2, 165-3 -> 충청남도 공주시 계룡면 화헌리 165-1, 충청남도 공주시 계룡면 화헌리 165-1, 충청남도 공주시 계룡면 화헌리 165-2, 충청남도 공주시 계룡면 화헌리 165-3
    '''
    dfc = df.copy()
    append_list = []
    ms_pattern = r'(산)?(\d+)?-?(\d+)?'
    ms_re = re.compile(ms_pattern)
    for x in tqdm(dfc.iterrows()):
        add_list = list() # 중복 제거를 위해 set
        x0, x1 = x[0], x[1]
        road_sd, jb_sd = x1.road_sd, x1.sd
        road_sg, jb_sg = x1.road_sg, x1.sg
        road_g, jb_g = x1.road_g, x1.g
        road_em, jb_em = x1.road_em, x1.em
        road_rg, jb_dlg = x1.road_rg, x1.dlg
        mainid, subid = x1.id[0], x1.id[1]

        # 지번 분할
        road_jb = sep_jb(x1.road_jb)
        jb = sep_jb(x1.jb)
        
        # 세종특별자치시 세종특별자치시 <- 와 같은 경우 시군 공백으로 치환
        if road_sd == road_sg:
            road_sg = ''
        if jb_sd == jb_sg:
            jb_sg = ''

        # 주소생성
        if len(road_jb) >= 1:

            for r in road_jb:
                ms = ms_re.findall(r)[0]
                mt, main_jb, sub_jb = ms[0], ms[1], ms[2]
                main_jb, sub_jb, _ = jb_ms_sep(main_jb, sub_jb, '')
                address, pnucd = road_to_jbaddress([road_sd, road_sg, road_g, road_em, main_jb+'-'+sub_jb, road_rg])
                if r:
                    add_list.append([mainid, subid, rm_space_str(road_sd+" "+road_sg+" "+road_g+" "+road_em+" "+road_rg+" "+r), address, 1, pnucd]) # 고유번호, sub_id, 생성주소, 변환주소, 도로명주소여부, PNU코드
        
        if len(jb) >= 1:
            save_main, post_main = '', ''
            for j in jb:
                if j:
                    ms = ms_re.findall(j)[0]
                    mt, main_jb, sub_jb = ms[0], ms[1], ms[2]
                    main_jb, sub_jb, post_main = jb_ms_sep(main_jb, sub_jb, post_main)
                    if main_jb:
                        save_main = main_jb
                    else:
                        main_jb, post_main = save_main, save_main
                    if not mt:
                        ms_yn = 0
                    else:
                        ms_yn = 1
                    if sub_jb and sub_jb != '0':
                        totaladd = jb_sd+" "+jb_sg+" "+jb_g+" "+jb_em+" "+jb_dlg+" "+mt+" "+main_jb+"-"+sub_jb
                    else:
                        totaladd = jb_sd+" "+jb_sg+" "+jb_g+" "+jb_em+" "+jb_dlg+" "+mt+" "+main_jb
                    add_list.append([mainid, subid, rm_space_str(totaladd), None, 0,address_pnu([jb_sd, jb_sg, jb_g, jb_em, jb_dlg, main_jb, sub_jb, ms_yn])]) # 고유번호, sub_id, 생성주소, 변환주소, 도로명주소여부, PNU코드

        append_list.extend(add_list)
        
    fdf = pd.DataFrame(append_list, columns=['고유번호', 'Sub_id', '생성주소', '변환주소', '도로명주소여부', 'PNU_CD'])
    fdf = fdf.drop_duplicates(subset=['고유번호','생성주소','변환주소','도로명주소여부','PNU_CD'])

    uniq_add_dict = dict()
    cnt = 0
    for add in fdf['생성주소'].unique():
        uniq_add_dict.setdefault(add, cnt)
        cnt += 1
    fdf['주소_id'] = fdf['생성주소'].apply(lambda x: uniq_add_dict.get(x))

    return fdf


col_nm = "FULL_ADDR"
bjd_cd = pd.read_csv('./csv/법정동코드 전체자료.csv')

df_energy = postal_code(df_energy, col_nm)
df_energy = sd_fullname(df_energy, col_nm)
df_energy = replace_exception_char(df_energy, col_nm)
df_energy = sep_name(df_energy, bjd_cd, col_nm)

dft = sep_address(df_energy, col_nm)
dft = address_supplement(dft)

dft_c = dft.copy()
fdf = make_multi_add(dft_c,)

In [25]:
df_rps = pd.read_csv("./csv/RPS_merge.csv", encoding = "cp949")
df_energy = pd.read_csv("./csv/신재생에너지_merge.csv", encoding = "cp949")

df_rps.rename(columns = {'고유번호' : '고유번호(rps)'}, inplace = True)
df_energy.rename(columns = {'고유번호' : '고유번호(농지전용)'}, inplace = True)

a = df_energy[['PNU_CD','고유번호(농지전용)']]
b = df_rps[['PNU_CD','고유번호(rps)']]

b['PNU_CD'] = b['PNU_CD'].astype(float)

a['PNU_CD'] = a['PNU_CD'].apply(lambda x: '%.0f' % x)
b['PNU_CD'] = b['PNU_CD'].apply(lambda x: '%.0f' % x)
df_pnu = pd.merge(a,b,on = "PNU_CD", how = "inner")

C:\Users\SUNDO\AppData\Local\Temp\ipykernel_3524\2600574606.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_rps = pd.read_csv("./csv/RPS_merge.csv", encoding = "cp949")


In [68]:
#불필요한 컬럼명 제거 + 농지전용인지 RPS인지 출처 밝히기
df_energy = df_energy.drop(['생성주소'], axis=1)
df_energy.rename(columns = {'주소_id' : '주소_id(농지전용)'}, inplace = True)
df_energy.rename(columns = {'Sub_id' : 'Sub_id(농지전용)'}, inplace = True)
df_energy.rename(columns = {'FULL_ADDR':'FULL_ADDR(농지전용)'}, inplace = True)

df_merge1 = pd.merge(df_pnu, df_energy, on = "고유번호(농지전용)", how = "inner")
df_merge1.rename(columns = {'도로명주소여부':'도로명주소여부(농지전용)'}, inplace = True)

df_rps = df_rps.drop(['Unnamed: 0','상세주소2','연간 평균 수입','PNU_CD'], axis=1)

df_rps.rename(columns = {'Sub_id' : 'Sub_id(RPS)'}, inplace = True)
df_rps.rename(columns = {'도로명주소여부' : '도로명주소여부(RPS)'}, inplace = True)
df_rps.rename(columns = {'주소_id' : '주소_id(RPS)'}, inplace = True)
df_rps.rename(columns = {'생성주소' : '생성주소(RPS)'}, inplace = True)
df_rps.rename(columns = {'변환주소' : '변환주소(RPS)'}, inplace = True)

df_merge2 = pd.merge(df_merge1, df_rps, on = "고유번호(rps)", how = "inner")
df_merge2.rename(columns = {'지목_x' : '지목(농지전용)'}, inplace = True)
df_merge2.rename(columns = {'지목_y' : '지목(RPS)'}, inplace = True)

df_merge2.to_csv("최종매칭DB(CP949)_v0.2.csv", encoding = "cp949")
df_merge2.to_csv("최종매칭DB(UTF-8)_v0.2.csv", encoding = "utf-8")

In [98]:
df_merge2.to_csv("최종매칭DB(CP949)_v0.2.csv", encoding = "cp949")
df_merge2.to_csv("최종매칭DB(UTF-8)_v0.2.csv", encoding = "utf-8")

In [99]:
df3 = pd.read_csv("./csv/최종매칭DB(CP949)_v0.2.csv", encoding = "cp949")

C:\Users\SUNDO\AppData\Local\Temp\ipykernel_3524\725642229.py:1: DtypeWarning: Columns (35,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("./csv/최종매칭DB(CP949)_v0.2.csv", encoding = "cp949")


In [101]:
df3.columns

Index(['Unnamed: 0', 'PNU_CD', '고유번호(농지전용)', '고유번호(rps)', '납부단체', '용도',
       '전용사업명', '필지시도명', '필지시군구', '농지소재지', '농지구분', '지목(농지전용)', '필지면적', '통지번호',
       '부과기준일', 'SIDO_GRP_SEQ', 'SIDO_SEQ', 'FULL_ADDR(농지전용)', 'Sub_id(농지전용)',
       '도로명주소여부(농지전용)', '주소_id(농지전용)', '시도', '시군', '상세주소1',
       '에너지원구분\n(신재생에너지원)', '업체(상호)명', '발전소명', '발전소수', '신규/증설/말소\n구분',
       '용량(kW)', '용량구분', '발전사업허가일', '설비확인\n연도', '설치형태', '지목(RPS)', '건축물용도',
       '구분', 'Sub_id(RPS)', '생성주소(RPS)', '변환주소(RPS)', '도로명주소여부(RPS)',
       '주소_id(RPS)'],
      dtype='object')